<a href="https://colab.research.google.com/github/MariaAise/acspri_llm/blob/main/code/into_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face in Practice: Multi-task Tutorial (Datasets + Transformers + Evaluate + Trackio + Gradio)

-----------
> In this course, you primarily interact with **Transformers** and **Datasets**.
>
> Other Hugging Face libraries exist to support scaling, evaluation, and fine-tuning, but they are not required for basic LLM usage.


## Hugging Face Libraries — API References

### 🤗 Transformers

**Purpose:** Pretrained models, pipelines, inference, fine-tuning

API documentation:
[https://huggingface.co/docs/transformers/index](https://huggingface.co/docs/transformers/index)

Pipelines (recommended entry point):
[https://huggingface.co/docs/transformers/main/en/pipeline_tutorial](https://huggingface.co/docs/transformers/main/en/pipeline_tutorial)

Python API reference:
[https://huggingface.co/docs/transformers/main/en/api_reference](https://huggingface.co/docs/transformers/main/en/api_reference)

---

### 🤗 Datasets

**Purpose:** Loading, processing, and managing text (and multimodal) datasets

API documentation:
[https://huggingface.co/docs/datasets/index](https://huggingface.co/docs/datasets/index)

Python API reference:
[https://huggingface.co/docs/datasets/main/en/package_reference](https://huggingface.co/docs/datasets/main/en/package_reference)

Loading datasets guide:
[https://huggingface.co/docs/datasets/loading](https://huggingface.co/docs/datasets/loading)

---

### 🤗 Tokenizers

**Purpose:** Fast tokenization backend used by Transformers (mostly implicit use)

API documentation:
[https://huggingface.co/docs/tokenizers/index](https://huggingface.co/docs/tokenizers/index)

Python API reference:
[https://huggingface.co/docs/tokenizers/python/latest/](https://huggingface.co/docs/tokenizers/python/latest/)

---

### 🤗 Evaluate

**Purpose:** Standardised evaluation metrics for NLP and ML tasks

API documentation:
[https://huggingface.co/docs/evaluate/index](https://huggingface.co/docs/evaluate/index)

Python API reference:
[https://huggingface.co/docs/evaluate/package_reference](https://huggingface.co/docs/evaluate/package_reference)

---

### 🤗 PEFT (Parameter-Efficient Fine-Tuning)

**Purpose:** LoRA and other lightweight fine-tuning methods

API documentation:
[https://huggingface.co/docs/peft/index](https://huggingface.co/docs/peft/index)

Python API reference:
[https://huggingface.co/docs/peft/package_reference](https://huggingface.co/docs/peft/package_reference)

(Used later when discussing fine-tuning conceptually.)

---

### 🤗 Hub (Model & Dataset Hosting)

**Purpose:** Model, dataset, and space hosting; authentication

Hub documentation:
[https://huggingface.co/docs/huggingface_hub/index](https://huggingface.co/docs/huggingface_hub/index)

Python API reference:
[https://huggingface.co/docs/huggingface_hub/package_reference](https://huggingface.co/docs/huggingface_hub/package_reference)

---

## Setup (Colab)

### 1. Install

In [1]:
!pip -q install -U datasets transformers evaluate accelerate gradio trackio sentence-transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 10.5 MB/s eta 0:00:00


### 2. Imports + device

In [2]:
import os, random, numpy as np
import torch

from datasets import load_dataset
from transformers import pipeline

import evaluate
import trackio as wandb  # Trackio is wandb-compatible. :contentReference[oaicite:0]{index=0}

DEVICE = 0 if torch.cuda.is_available() else -1
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

print("GPU available:", torch.cuda.is_available(), "DEVICE:", DEVICE)

GPU available: False DEVICE: -1


### 0.3 Trackio: initialize a project

Trackio logs runs locally by default and you can launch a local dashboard via `trackio show`.

In [3]:
PROJECT = "hf-multitask-workshop"

def start_run(run_name: str, config: dict):
    wandb.init(project=PROJECT, name=run_name, config=config)

def log_metrics(metrics: dict, step: int | None = None):
    if step is None:
        wandb.log(metrics)
    else:
        wandb.log(metrics | {"step": step})

def end_run():
    wandb.finish()

## Working with datasets

Working with good data is the key, especially at early stage as you learn to work with Python.

Hugging Face provides access to some best quality classic datasets that allow you to learn in the smoothest way.

Data is available across all the modalities, such as text, image, audio, video as well as Hugging Face provides convenient tools to work with the data.


[DatasetInfo](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.DatasetInfo): provides general info about datasets and is a good place to start

Datasets can be heavy and committing to downloading them can be challenging: Use the `load_dataset_builder()` function to load a dataset builder and inspect a dataset’s attributes without committing to downloading it:

In [4]:
from datasets import load_dataset_builder
ds_builder = load_dataset_builder("cornell-movie-review-data/rotten_tomatoes")

ds_builder.info.description

ds_builder.info.features

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}

**Splits**
A split is a specific subset of a dataset like train and test. List a dataset’s split names with the get_dataset_split_names() function:

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names("cornell-movie-review-data/rotten_tomatoes")

Then you can load a specific split with the split parameter. Loading a dataset split returns a Dataset object:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")
dataset

If you don’t specify a split, 🤗 Datasets returns a DatasetDict object instead:


In [ ]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

Some datasets contain several sub-datasets. For example, the `MInDS-14` dataset has several sub-datasets, each one containing audio data in a different language. These sub-datasets are known as configurations or subsets, and you must explicitly select one when loading the dataset. If you don’t provide a configuration name, 🤗 Datasets will raise a ValueError and remind you to choose a configuration.

Use the `get_dataset_config_names()` function to retrieve a list of all the possible configurations available to your dataset:

In [ ]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("PolyAI/minds14")
print(configs)

In [ ]:
from datasets import load_dataset

mindsFR = load_dataset("PolyAI/minds14", "fr-FR", split="train")

### Iterable vs regular datasets

There are two types of dataset objects, a **regular Dataset** and then an ✨ **IterableDataset** ✨.

A Dataset provides fast **random** access to the rows, and memory-mapping so that loading even large datasets only uses a relatively small amount of device memory. But for really, really big datasets that won’t even fit on disk or in memory, an IterableDataset allows you to access and use the dataset without waiting for it to download completely.

### Dataset

When you load a dataset split, you’ll get a `Dataset` object.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")

**Indexing**

A Dataset contains columns of data, and each column can be a different type of data. The **index**, or **axis label**, is used to access examples from the dataset.

In [ ]:
dataset[0]

to access the last one: -


In [ ]:
dataset[-1]

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

Indexing by the column name returns a list of all the values in the column:

In [ ]:
dataset[0]["text"]

You can combine row and column name indexing to return a specific value at a position:

In [ ]:
dataset[0]["text"]

**Slicing**

Slicing returns a slice - or subset - of the dataset, which is useful for viewing several rows at once. To slice a dataset, use the : operator to specify a range of positions.

In [ ]:
dataset[:3]

dataset[3:6]

#### IterableDataset

An IterableDataset is loaded when you set the streaming parameter to True in load_dataset():

In [ ]:
from datasets import load_dataset

iterable_dataset = load_dataset("ethz/food101", split="train", streaming=True)
for example in iterable_dataset:
    print(example)
    break

In [ ]:
An IterableDataset’s behavior is different from a regular Dataset. You don’t get random access to examples in an IterableDataset. Instead, you should iterate over its elements, for example, by calling next(iter()) or with a for loop to return the next item from the IterableDataset:




In [ ]:
next(iter(iterable_dataset))

for example in iterable_dataset:
    print(example)
    break

---
## HF Tasks
---

## 1) Text classification (AG News) + Evaluate + Trackio

**Dataset:** `ag_news`
**Task:** supervised text classification (4 classes)

### 1.1 Load a small slice

In [ ]:
ds = load_dataset("ag_news")
print(ds)

# Small slices for speed
train_small = ds["train"].shuffle(seed=42).select(range(600))
test_small  = ds["test"].shuffle(seed=42).select(range(300))

label_names = ds["train"].features["label"].names
label_names

### 1.2 Baseline model (no training): `pipeline("text-classification")`

Pick a fast checkpoint that supports 4-way news classification. This one is commonly used for AG News:

In [ ]:
MODEL = "textattack/bert-base-uncased-ag-news"

clf = pipeline(
    task="text-classification",
    model=MODEL,
    tokenizer=MODEL,
    device=DEVICE,
    truncation=True,
    max_length=256,
)

### 1.3 Predict + evaluate

In [ ]:
acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def predict_batch(texts):
    out = clf(texts, batch_size=16)
    # pipeline returns labels like "LABEL_0"... map to id
    pred_ids = [int(o["label"].split("_")[-1]) for o in out]
    scores   = [float(o["score"]) for o in out]
    return pred_ids, scores

texts = test_small["text"]
y_true = test_small["label"]

y_pred, y_score = predict_batch(texts)

metrics = {
    "accuracy": acc.compute(predictions=y_pred, references=y_true)["accuracy"],
    "f1_macro": f1.compute(predictions=y_pred, references=y_true, average="macro")["f1"],
    "mean_confidence": float(np.mean(y_score)),
}
metrics

### 1.4 Log to Trackio

In [ ]:
start_run(
    run_name="ag_news_baseline_inference",
    config={"dataset":"ag_news", "model":MODEL, "max_length":256, "batch_size":16}
)

log_metrics(metrics)

# Log a few examples as a lightweight artifact pattern
examples = []
for i in range(5):
    examples.append({
        "text": texts[i][:280],
        "true": label_names[y_true[i]],
        "pred": label_names[y_pred[i]],
        "score": y_score[i],
    })
wandb.log({"examples": examples})

end_run()
print("Run logged. To view dashboard in a terminal: trackio show")  # :contentReference[oaicite:2]{index=2}

---

## 2) Zero-shot text classification (AG News → your own labels)

**Task:** zero-shot classification (no labels in training; you provide candidate labels)

### 2.1 Build a zero-shot pipeline

In [ ]:
ZSHOT_MODEL = "facebook/bart-large-mnli"

zshot = pipeline(
    task="zero-shot-classification",
    model=ZSHOT_MODEL,
    device=DEVICE,
)

### 2.2 Define labels and run

In [ ]:
candidate_labels = ["politics", "business", "sports", "technology"]

sample_texts = test_small["text"][:20]
out = zshot(sample_texts, candidate_labels=candidate_labels, batch_size=8)

pred_labels = [o["labels"][0] for o in out]
pred_scores = [float(o["scores"][0]) for o in out]

list(zip(pred_labels[:5], pred_scores[:5]))

### 2.3 Evaluate by mapping your labels → dataset labels

If you align labels to AG News classes, you can evaluate. Here is one simple mapping:

In [ ]:
# AG News labels: ["World","Sports","Business","Sci/Tech"]
ag_to_custom = {
    "World": "politics",
    "Sports": "sports",
    "Business": "business",
    "Sci/Tech": "technology",
}
custom_to_ag = {v:k for k,v in ag_to_custom.items()}

y_true_names = [label_names[i] for i in test_small["label"][:20]]
y_true_custom = [ag_to_custom[n] for n in y_true_names]
acc_custom = sum(p==t for p,t in zip(pred_labels, y_true_custom)) / len(y_true_custom)
acc_custom

### 2.4 Log the run

In [ ]:
start_run(
    run_name="ag_news_zero_shot",
    config={"dataset":"ag_news", "model":ZSHOT_MODEL, "candidate_labels":candidate_labels}
)
log_metrics({"zero_shot_acc_on_20": acc_custom, "mean_top_score": float(np.mean(pred_scores))})
end_run()

---

## 3) Multi-label emotion (GoEmotions) with a zero-shot baseline

**Dataset:** `go_emotions`
**Task:** multi-label emotion (harder; good for “why evaluation matters”)

### 3.1 Load a small slice

In [ ]:
go = load_dataset("go_emotions", "raw")
go_small = go["test"].shuffle(seed=42).select(range(120))

# Dataset has many labels; for a clean demo pick a manageable subset
emotion_labels = ["joy", "sadness", "anger", "fear", "surprise", "disgust"]

### 3.2 Zero-shot multi-label trick

Zero-shot pipeline supports `multi_label=True`.

In [ ]:
texts = go_small["text"]
out = zshot(texts, candidate_labels=emotion_labels, multi_label=True, batch_size=8)

# Keep top-2 predicted emotions per text (demo choice)
pred_top2 = [o["labels"][:2] for o in out]
pred_top2[:3]

### 3.3 Log qualitative outputs (this is the point)

In [ ]:
start_run(
    run_name="go_emotions_zero_shot_top2",
    config={"dataset":"go_emotions", "model":ZSHOT_MODEL, "labels":emotion_labels, "topk":2}
)
wandb.log({"examples": [{"text": texts[i][:240], "pred_top2": pred_top2[i]} for i in range(10)]})
end_run()

---

## 4) Image classification (CIFAR-10) + Gradio demo

**Dataset:** `cifar10`
**Task:** image classification (instant visual engagement)

### 4.1 Load a small slice

In [ ]:
cifar = load_dataset("cifar10")
cifar_small = cifar["test"].shuffle(seed=42).select(range(60))
cifar_labels = cifar["train"].features["label"].names
cifar_labels

### 4.2 Pipeline

In [ ]:
IMG_MODEL = "google/vit-base-patch16-224"

img_clf = pipeline(
    task="image-classification",
    model=IMG_MODEL,
    device=DEVICE
)

# quick sanity
img = cifar_small[0]["img"]
img_clf(img)[:3]

### 4.3 Gradio app (image → top-5 labels)

In [ ]:
import gradio as gr

def classify_image(image):
    preds = img_clf(image)
    # return as nice table
    return {p["label"]: float(p["score"]) for p in preds[:5]}

demo1 = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=5),
    title="Image Classification (ViT)",
    description="Upload an image (CIFAR-10 works best for this demo)."
)

demo1.launch(share=True)

---

## 5) Image captioning (Pokémon BLIP captions dataset) + “WOW” demo

**Dataset:** `lambdalabs/pokemon-blip-captions`
**Task:** image → text captioning (high WOW, low effort)

### 5.1 Load a small slice

In [ ]:
poke = load_dataset("lambdalabs/pokemon-blip-captions", split="train")
poke_small = poke.shuffle(seed=42).select(range(80))

poke_small[0].keys()

### 5.2 Caption pipeline

In [ ]:
CAPTION_MODEL = "Salesforce/blip-image-captioning-base"

captioner = pipeline(
    task="image-to-text",
    model=CAPTION_MODEL,
    device=DEVICE
)

# Try one image
captioner(poke_small[0]["image"])[0]

### 5.3 Gradio app (image → caption)

In [ ]:
def caption_image(image):
    out = captioner(image, max_new_tokens=30)
    return out[0]["generated_text"]

examples = [[poke_small[i]["image"]] for i in range(6)]

demo2 = gr.Interface(
    fn=caption_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(label="Caption"),
    examples=examples,
    title="Image Captioning (BLIP)",
    description="Upload an image or click an example (Pokémon dataset)."
)
demo2.launch(share=True)

### 5.4 Log a run (caption quality is qualitative)

In [ ]:
start_run(
    run_name="pokemon_captioning_blip",
    config={"dataset":"lambdalabs/pokemon-blip-captions", "model":CAPTION_MODEL, "n_examples":10}
)
samples = []
for i in range(10):
    cap = captioner(poke_small[i]["image"], max_new_tokens=30)[0]["generated_text"]
    samples.append({"dataset_caption": poke_small[i]["text"], "model_caption": cap})
wandb.log({"caption_samples": samples})
end_run()

---

## 6) Multimodal retrieval (text ↔ image) using CLIP embeddings

**Goal:** Type a text query → retrieve nearest images (very strong “magic” moment).

### 6.1 Embed images + captions (small index)

In [ ]:
from sentence_transformers import SentenceTransformer
from PIL import Image

CLIP_MODEL = "clip-ViT-B-32"
clip = SentenceTransformer(CLIP_MODEL)

# Build a tiny retrieval index
N = 120
idx_ds = poke.shuffle(seed=0).select(range(N))

images = [ex["image"] for ex in idx_ds]
captions = [ex["text"] for ex in idx_ds]

# Encode images and text into same embedding space
img_emb = clip.encode(images, batch_size=16, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
txt_emb = clip.encode(captions, batch_size=32, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

img_emb.shape, txt_emb.shape

### 6.2 Retrieval function

In [ ]:
def retrieve_images_by_text(query: str, k: int = 6):
    q = clip.encode([query], normalize_embeddings=True, convert_to_numpy=True)[0]
    scores = img_emb @ q  # cosine similarity (normalized)
    topk = np.argsort(-scores)[:k]
    return [images[i] for i in topk], [float(scores[i]) for i in topk], [captions[i] for i in topk]

### 6.3 Gradio app (text → gallery)

In [ ]:
def gr_retrieve(query):
    imgs, scores, caps = retrieve_images_by_text(query, k=8)
    gallery = []
    for im, s, c in zip(imgs, scores, caps):
        gallery.append((im, f"{s:.3f} | {c[:60]}"))
    return gallery

demo3 = gr.Interface(
    fn=gr_retrieve,
    inputs=gr.Textbox(label="Text query", placeholder="e.g., 'yellow mouse with lightning'"),
    outputs=gr.Gallery(label="Top matches", columns=4, height="auto"),
    title="Text → Image Retrieval (CLIP)",
    description="CLIP embeds text and images into the same space; nearest neighbors are semantically related."
)
demo3.launch(share=True)

### 6.4 Trackio logging for retrieval (simple Recall@K demo)

A quick, lightweight metric: for each dataset caption, does its own image appear in top-K when you query the caption?


In [ ]:
def recall_at_k(k=5, n=60):
    hits = 0
    for i in range(n):
        query = captions[i]
        q = clip.encode([query], normalize_embeddings=True, convert_to_numpy=True)[0]
        scores = img_emb @ q
        topk = set(np.argsort(-scores)[:k])
        if i in topk:
            hits += 1
    return hits / n

r5 = recall_at_k(k=5, n=60)
r10 = recall_at_k(k=10, n=60)

start_run(
    run_name="pokemon_clip_retrieval",
    config={"dataset":"lambdalabs/pokemon-blip-captions", "clip_model":CLIP_MODEL, "index_size":N}
)
log_metrics({"recall@5": r5, "recall@10": r10})
end_run()

## 7) Trackio dashboard

From a terminal (or Colab cell with `!`):

In [ ]:
!trackio show

In [ ]:
# Optionally open a specific project:

!trackio show --project "hf-multitask-workshop"

>Note

#### Weights & Biases (wandb): experiment tracking

**What it is**
A service + Python library for logging and comparing ML/LLM experiments (“runs”).

**What you use it for in this course context**

* Record parameters (model name, prompt version, temperature, dataset slice)
* Log metrics (accuracy/F1, cost per 1k tokens, latency, hallucination rate)
* Save artifacts (datasets, prompt templates, evaluation outputs)
* Compare runs across prompt variants or model choices

**Typical workflow**

1. Start a run
2. Log config + metrics during processing
3. Review runs in the web UI to compare results and reproduce

**Minimal example**

```python
!pip -q install wandb
import wandb

wandb.login()  # opens auth flow

wandb.init(project="acspri-llm-tutorial", config={
    "model": "gemini-1.5-flash",
    "task": "sentiment",
    "temperature": 0.2
})

wandb.log({"accuracy": 0.81, "avg_latency_s": 0.42, "usd_cost_est": 0.18})
wandb.finish()
```

**When to use**

* Any time you run multiple prompt/model variants and need a record of what changed.
* Any time you need reproducibility for a report/paper.

---

## Trackio: lightweight local tracking (no hosted dashboard required)

**What it is**
A small local-first tracking library for logging runs and metrics without relying on an external hosted platform.

**What you use it for in this course context**

* Track prompt/model variants locally inside Colab
* Log metrics and outputs to files you can save to Drive/GitHub
* Keep an audit trail for research workflows when external services are not allowed

**Typical workflow**

1. Create a local run directory
2. Log config + metrics + outputs to JSON/CSV
3. Save the folder to Drive or commit to GitHub

**Minimal “trackio-style” pattern (local run folder)**
If you do not want to depend on any particular tracking vendor, teach this neutral pattern:

```python
import json, time
from pathlib import Path

RUNS = Path("/content/outputs/runs")
RUNS.mkdir(parents=True, exist_ok=True)

run_id = time.strftime("%Y%m%d-%H%M%S")
run_dir = RUNS / run_id
run_dir.mkdir()

config = {"model": "gemini-1.5-flash", "temperature": 0.2, "task": "sentiment"}
(run_dir / "config.json").write_text(json.dumps(config, indent=2))

metrics = {"accuracy": 0.81, "avg_latency_s": 0.42, "usd_cost_est": 0.18}
(run_dir / "metrics.json").write_text(json.dumps(metrics, indent=2))

print("Saved run to:", run_dir)
```

**When to use**

* When you need tracking but cannot use hosted tools.
* When you want outputs you can bundle as research artifacts (Drive/GitHub).

---